In [2]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

In [3]:
# Importing dataset
dataset = pd.read_excel("Data//log.xlsx")

In [4]:
# Display dataset
print(dataset)
# 1509 data point, 11 columns (10 logs + depth)

       DEPTH    GR    SP   Rt  Rxo  DEN  SONIC   CNL   DIP   AZIM  MUDLOG
0     5195.0  91.4 -22.7  2.2  4.3  2.0  103.8  36.2  10.1  300.5       1
1     5195.5  96.1 -25.0  3.4  6.9  2.0  104.3  36.7  10.5  297.1       1
2     5196.0  95.0 -25.0  2.7  5.4  2.0  101.5  38.8   8.1  303.4       1
3     5196.5  95.2 -23.1  2.9  5.5  2.0  103.8  36.6   7.8  296.2       1
4     5197.0  92.8 -26.2  2.7  5.7  2.1  109.5  36.8   7.5  299.2       1
...      ...   ...   ...  ...  ...  ...    ...   ...   ...    ...     ...
1504  5947.0  32.5 -22.9  4.1  7.7  2.4   77.5  12.5   4.7  285.8       0
1505  5947.5  31.3 -20.8  4.0  7.3  2.5   85.3  11.8   4.2  291.9       0
1506  5948.0  32.6 -22.3  4.2  7.8  2.3   78.0  11.6   5.8  298.0       0
1507  5948.5  35.0 -21.3  4.0  7.5  2.5   78.8  12.4   5.4  295.5       0
1508  5949.0  32.1 -23.5  3.9  7.5  2.4   84.2  11.7   7.0  294.8       0

[1509 rows x 11 columns]


In [5]:
cols = ['GR','SP','Rt','Rxo','DEN','SONIC','CNL','DIP']
# input data
input_dataset = dataset[cols]
# 1000 data inputs for taining
train_dataset = np.array(input_dataset[0:1000], dtype = np.float32)
# 509 data inputs for testing the model
test_dataset = np.array(input_dataset[1000:], dtype = np.float32)

# output data
labels = dataset['MUDLOG']
# output data for training
train_labels = torch.from_numpy(np.array(labels[0:1000], dtype = np.float32))
# output data for testing
test_labels = torch.from_numpy(np.array(labels[1000:], dtype = np.float32))

# To solve the randomness issue
seed = 4
torch.manual_seed(seed)

In [6]:
# Create model object
class FNNmodel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FNNmodel, self).__init__()
        # Linear -> NonLinear -> Linear
        ''' First Layer      dimensions: 8 -> 7 '''
        #linear function - FC1
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        #nonlinear function
        self.relu1 = nn.ReLU() 
        ''' Readout Layer    dimensions: 7 -> 1 '''
        # linear function - FC2
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Sigmoid func to convert resutls into 0 and 1
        self.sig = nn.Sigmoid()
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.sig(self.fc2(out))
        return out

In [7]:
# Determine input, output dimensions 
# and number of hidden layers
input_dim = 1*8
hidden_dim = 7
output_dim = 1
# Initiate model class
model = FNNmodel(input_dim, hidden_dim, output_dim)

In [8]:
# Display dimensions of the model parameters

print(model.parameters())

print(len(list(model.parameters())))

# Hidden layer parameters
print(list(model.parameters())[0].size())

# FC1 Bias parameters
print(list(model.parameters())[1].size())

# FC2 parameters
print(list(model.parameters())[2].size())

# FC2 Bias parameters
print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x7f6fda2cb430>
4
torch.Size([7, 8])
torch.Size([7])
torch.Size([1, 7])
torch.Size([1])


In [14]:
# Evaluating the model/Shows layers and their activation function summary
model.eval()

FNNmodel(
  (fc1): Linear(in_features=8, out_features=7, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=7, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [10]:
# loss function
criterion = nn.BCELoss()

In [11]:
# learning rate and optimizer class
learning_rate = 0.0001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
# num of epochs
epochs = 4000

# Training and testing the model
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(train_dataset)
    labels = train_labels.view(-1, 1)
    labels = labels.type(torch.float32)
    # Reset gradients to zero
    optimizer.zero_grad()
    # Applying funtions (model)
    outputs = model(inputs)
    # Calculate loss
    loss = criterion(outputs, labels)
    # Calculate gradients
    loss.backward()
    # Update Parameters
    optimizer.step()
    
    # Calculate Accuracy
    if epoch%400 == 0:
        inputs = torch.from_numpy(test_dataset)
        labels = test_labels.view(-1, 1)
        labels = labels.type(torch.float32)
        correct = 0
        total = 0
        for _ in range(400):
            outputs = model(torch.from_numpy(test_dataset)).data
            predicted = np.round(outputs.data)
            total += labels.size(0)

            correct += (predicted == labels).sum()
        accuracy = 100*correct/total
        # Display loss and accuray
        print('epoch = {}, loss = {}, accuracy = {}'.format(epoch, loss.data, accuracy))

epoch = 400, loss = 0.22564241290092468, accuracy = 94.30255126953125
epoch = 800, loss = 0.21018216013908386, accuracy = 94.49901580810547
epoch = 1200, loss = 0.19729095697402954, accuracy = 94.49901580810547
epoch = 1600, loss = 0.18596310913562775, accuracy = 94.49901580810547
epoch = 2000, loss = 0.17580272257328033, accuracy = 94.49901580810547
epoch = 2400, loss = 0.16665557026863098, accuracy = 94.49901580810547
epoch = 2800, loss = 0.15832555294036865, accuracy = 94.69548034667969
epoch = 3200, loss = 0.1506505012512207, accuracy = 94.69548034667969
epoch = 3600, loss = 0.1435127556324005, accuracy = 95.48133850097656
epoch = 4000, loss = 0.13682463765144348, accuracy = 95.67780303955078


In [13]:
torch.save(model.state_dict(), "Saved Models/FNN")